# Flight Assistant with Gradio and OpenAI
 Before executing the code, ensure that the initial setup outlined in the Readme.md file has been completed.

In [20]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

import base64
from io import BytesIO
from PIL import Image

from pydub import AudioSegment
from pydub.playback import play

In [21]:
import os
os.environ['PATH'] = '/opt/homebrew/bin:' + os.environ['PATH']

In [22]:
# Constants

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [23]:
#Prompt Setup:

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [24]:
# Model Execution

"""
We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format """

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [25]:
# Visualization
# Adding share=True means that it can be accessed publically.NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.
# Adding inbrowser=True opens up a new browser window automatically
gr.ChatInterface(fn=chat, type="messages").launch(share=True,inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://da0220ab752704a559.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#### Tool

In [26]:
#Tool
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

#print(get_ticket_price("Berlin"))

# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# This is included in a list of tools the model can use:
tools = [{"type": "function", "function": price_function}]

In [27]:
# Model Execution with tool: give the LLM the opportunity to inform us that it wants us to run the tool.

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content


def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [28]:
# Visualization dark mode

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch(share=True,inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://bb9aaa700bb5c06389.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#### Let's go multi-modal!!

We use DALL-E-3, the image generation model behind GPT-4o, to make us some images.Let's put this in a function called artist.

Warning: Price alert, each time I generate an image it costs about 4c - don't go crazy with images!

In [29]:
# Model Execution: Image
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024", #the min size at the moment
            n=1, #cuantas respuestas
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [30]:
#Test image function
#image = artist("New York City")
#display(image)

In [31]:
# Model Execution: Audio
def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Type of voice
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [32]:
#Test Audio Funtion
#talker("Well, hi there")

Input #0, wav, from '/var/folders/x_/gjdjch3n18b_znsmwt0_wm2w0000gn/T/tmpdzxw12b2.wav':
  Duration: 00:00:00.77, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s


#### Agent Framework
The term 'Agentic AI' and Agentization is an umbrella term that refers to a number of techniques, such as:

1. Breaking a complex problem into smaller steps, with multiple LLMs carrying out specialized tasks
2. The ability for LLMs to use Tools to give them additional capabilities
3. The 'Agent Environment' which allows Agents to collaborate
4. An LLM can act as the Planner, dividing bigger tasks into smaller ones for the specialists
5. The concept of an Agent having autonomy / agency, beyond just responding to a prompt - such as Memory

We're showing 1 and 2 here

In [33]:
# Model Execution: Audio
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(city)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [34]:
#Visualization
# More involved Gradio code as we're not using the preset Chat interface!

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Paris


Input #0, wav, from '/var/folders/x_/gjdjch3n18b_znsmwt0_wm2w0000gn/T/tmp4zu7cafy.wav':
  Duration: 00:00:03.02, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s


Input #0, wav, from '/var/folders/x_/gjdjch3n18b_znsmwt0_wm2w0000gn/T/tmp0q9pq0oc.wav':
  Duration: 00:00:07.18, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
